In [2]:
import cv2
import numpy as np
import threading
from ultralytics import YOLO

video_paths = ["segment_1.mp4", "segment_2.mp4", "segment_3.mp4", "segment_4.mp4"]
model = YOLO("Hardhat_Jun03.pt")

def process_video(video_path, output_name):
    print(f"Processing {video_path}...")
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Failed to open {video_path}")
        return

    save_output = True
    if save_output:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_name, fourcc, int(cap.get(5)),
                              (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        results = model.track(frame, persist=True)
        result = results[0]
        frame_out = frame.copy()

        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
            cls_id = int(box.cls[0])
            tid = int(box.id[0]) if box.id is not None else 0
            conf = float(box.conf[0])
            color = (0, 255, 0) if cls_id == 0 else (0, 0, 255)
            label = f"{model.names[cls_id]} #{tid}"
            cv2.rectangle(frame_out, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame_out, f"Conf: {conf:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

        if save_output:
            out.write(frame_out)

        cv2.imshow(f"{video_path}", cv2.resize(frame_out, (800, 600)))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    if save_output:
        out.release()
    cv2.destroyWindow(f"{video_path}")
    print(f"Completed {video_path}")

threads = []
for i, vid in enumerate(video_paths):
    output_name = f"output_{i+1}.mp4"
    t = threading.Thread(target=process_video, args=(vid, output_name))
    t.start()
    threads.append(t)

for t in threads:
    t.join()

cv2.destroyAllWindows()
print("All videos processed.")

Processing segment_1.mp4...
Processing segment_2.mp4...
Processing segment_3.mp4...
Processing segment_4.mp4...

Ultralytics 8.3.156  Python-3.11.9 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)


Ultralytics 8.3.156  Python-3.11.9 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Ultralytics 8.3.156  Python-3.11.9 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

Model summary (fused): 92 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs
Model summary (fused): 92 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs
Model summary (fused): 92 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs
0: 384x640 1 Hardhat, 3 No_hardhats, 194.0ms
Speed: 4.1ms preprocess, 194.0ms inference, 145.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Hardhats, 1 No_hardhat, 180.1ms
Speed: 4.1ms preprocess, 180.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Hardhat, 1 No_har